# Image Processing

This Jupyter notebook demonstrates how to import images (RAW and other formats) and do spacial Fourier transforms to obtain and display spacial power spectra.

* Press `shift`-`enter` or `shift`-`return` when the cursor in in a cell or the cell is selected to run the code in the cell.
* Run the code cells in order for this demonstration.
* Remember that the order in which you run the cells is important, not the order they appear in the notebook.
* Use this code as an example. Change the files and parameters to fit your needs.

## Import Modules

The cell below to import the numerical and graphing modules and to configure the graphics interface.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

The next cell loads image modules.

In [0]:
import rawpy
import PIL

## Importing RAW Images

The cell below imports the RAW file named `'APC_0005.dng'` into a variable named `rgb`.

In [0]:
raw = rawpy.imread('APC_0005.dng')
rgb = raw.postprocess(use_camera_wb=True)
raw.close()

In [0]:
plt.imshow(rgb)

## Importing JPEG and Other Images

We use a different function to import JPEG, PNG, TIFF, or other images.

In [0]:
im = PIL.Image.open('TigerLilly.jpg')
xmin, ymin, xmax, ymax = im.getbbox()
imdata_rgb = np.array(im.getdata()).reshape(ymax-ymin, xmax-xmin, 3)
im.close()

In [0]:
plt.imshow(imdata_rgb)

## Image Calculations

Now we are going to do some calculations with the the imported RAW image, which is stored in the variable `rgb`.

The cell below gives the shape of `rgb`. The first index is the y size. The second is the x size. The third is the number of color channels (3 for red, green, and blue).

In [0]:
rgb.shape

The cell below stores the size of the x and y dimensions into variables (with shorter names) so we can use them later.

In [0]:
Nx = rgb.shape[1]
Ny = rgb.shape[0]

Notice in the RAW section above, the origin of the image by default is in the upper left corner. This if different than our regular convention for the y values to increase in the upward direction. For convenience, we will create a new variable with the y-index reversed so the origin is in the lower left corner and the y-values increase in the upward direction. Note that this step is not necessary. All out subsequent calculations will work otherwise, but we want the coordinate system to make sense.

The cell below creates a new variable `pic` with the y-index reversed.

In [0]:
pic = rgb[::-1,:]

The cell below displays `pic` with the origin in the lower left corner. Notice the y values are now in the standard order.

In [0]:
plt.imshow(pic, origin='lower')

### Red Channel

The cell below plots the red channel of `pic`. The red channel is the first component in the last (color) dimension of `pic`. Remember tht in Python the first componnt of an array or list is number 0. We use a red color scale in the image below to represent the red channel.

In [0]:
plt.imshow(pic[:,:,0], cmap='Reds', origin='lower')

Now we take a 2-D Fourier teansform (transform in both the x and y directions) of the red channel.

In [0]:
red_fft = np.fft.fft2(pic[:,:,0])

The cell below calculates the frequency values in the Fourier transform we just did. (We will use these same frequency values for the green and blue channels as well.) Since this is a spacial Fourier transform (the data is a function of position, not time), the frequncy values are called *wavenumbers*.

In [0]:
xfreqs = np.fft.fftfreq(Nx)
yfreqs = np.fft.fftfreq(Ny)

The two cells below calculate the minimum and maximum power spectrum values for the signal. We take the logarithm of the power spectrum to compress the scale, much like we did for sound.

In [0]:
np.min(20*np.log10(np.abs(red_fft)))

In [0]:
np.max(20*np.log10(np.abs(red_fft)))

The cell below shows the red channel spacial power spectrum. We use a decibel scale to compress the range of values much like we did with sound data. We use the standard color map (viridis) instead of the red color map because it is better at displaying a differences in values over a large range. We set the minimum and maximum values on the color scale with the `vmin` and `vmax` parameters in the `ax.imshow` command. (Try setting them to different values.) Notice that most of the large values are near the origin.

In [0]:
fig, ax = plt.subplots(figsize=(10,20))
cax = ax.imshow(20*np.log10(np.abs(red_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), 
                vmin=60, vmax=140)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('Red Channel')
fig.colorbar(cax, shrink=0.4, label='PSD (dB)');

The cell below displays the same image as the previous cell, but it compresses the color scale so the details are visible. Notce the diagonal line of high values in the lower left of the image, which is at the same angle and the light and dark pattern. We are using a higher resolution setting, `dpi=300`, to see more detail.

In [0]:
fig, ax = plt.subplots(figsize=(10,20), dpi=300)
cax = ax.imshow(20*np.log10(np.abs(red_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), 
                vmin=60, vmax=100)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('Red Channel')
fig.colorbar(cax, shrink=0.4, label='PSD (dB)');

### Green Channel

The cell below displays the green channel with a green color map.

In [0]:
plt.imshow(pic[:,:,1], cmap='Greens', origin='lower')

The rest of the cells in this section perform the same calculations done for the red channel on the green channel.

In [0]:
green_fft = np.fft.fft2(pic[:,:,1])

In [0]:
np.min(20*np.log10(np.abs(green_fft)))

In [0]:
np.max(20*np.log10(np.abs(green_fft)))

In [0]:
fig, ax = plt.subplots(figsize=(10,20))
cax = ax.imshow(20*np.log10(np.abs(green_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), 
                vmin=60, vmax=140)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('Green Channel')
fig.colorbar(cax, shrink=0.4, label='PSD (dB)');

In [0]:
fig, ax = plt.subplots(figsize=(10,20), dpi=300)
cax = ax.imshow(20*np.log10(np.abs(green_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), 
                vmin=60, vmax=100)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('Green Channel')
fig.colorbar(cax, shrink=0.4, label='PSD (dB)')

### Blue Channel

The cell below displays the blue channel with a blue color map.

In [0]:
plt.imshow(pic[:,:,1], cmap='Blues', origin='lower')

The rest of the cells in this section perform the same calculations done for the red and green channels on the blue channel.

In [0]:
blue_fft = np.fft.fft2(pic[:,:,2])

In [0]:
np.min(20*np.log10(np.abs(blue_fft)))

In [0]:
np.max(20*np.log10(np.abs(blue_fft)))

In [0]:
fig, ax = plt.subplots(figsize=(10,20))
cax = ax.imshow(20*np.log10(np.abs(blue_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), 
                vmin=60, vmax=140)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('Blue Channel')
fig.colorbar(cax, shrink=0.4, label="log(PSD)");

In [0]:
fig, ax = plt.subplots(figsize=(10,20), dpi=300)
cax = ax.imshow(20*np.log10(np.abs(blue_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), 
                vmin=60, vmax=95)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('Blue Channel')
fig.colorbar(cax, shrink=0.4, label='PSD (dB)');

## Three Channels Combined (Grayscale)

In [0]:
pic.shape

In [0]:
sum_image = np.array([], dtype=np.uint16)
sum_image = np.uint16(pic[:,:,0]) + np.uint16(pic[:,:,1]) + np.uint16(pic[:,:,2])

In [0]:
sum_image.shape

In [0]:
fig, ax = plt.subplots()
cax = ax.imshow(sum_image, origin="lower", cmap="gray")
fig.colorbar(cax, shrink=0.4, label='Summed Pixel Value');

In [0]:
sum_fft = np.fft.fft2(sum_image)

In [0]:
fig, ax = plt.subplots(figsize=(10,20), dpi=300)
cax = ax.imshow(20*np.log10(np.abs(sum_fft[:Ny//2+1,:Nx//2+1])), origin='lower', 
                extent=(xfreqs[0], -xfreqs[Nx//2], yfreqs[0], -yfreqs[Ny//2]), vmin=90, vmax=120)
ax.set_xlabel('x Wavenumber')
ax.set_ylabel('y Wavenumber')
ax.set_title('All Channels')
fig.colorbar(cax, shrink=0.4, label='PSD (dB)');